QUESTION 4-- How does general market sentiment on wsb correspond to the S&P 500

In [ ]:
import pandas as pd
import plotly.express as px 
from scipy.stats import pearsonr, chi2_contingency
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, date

In [ ]:
wsb = pd.read_csv('wsbData.csv')
wsb['date'] = pd.to_datetime(wsb['date']).dt.date

 
def wordsDay(df):
        words = ""
        for word in df['title']:
                words  = words + " " + word
        for word in df['content']:
                words  = words + " " + str(word)
        return words

In [ ]:
#  returns the positive sentiment intensity for a sentence
def pos_score(sentence: str):
    sentence_analyzer = SentimentIntensityAnalyzer()
    temp_sentence = sentence
    sentiment_result = sentence_analyzer.polarity_scores(temp_sentence)
    return (sentiment_result['pos'])

# returns the negative sentiment intensity for a sentence
def neg_score(sentence: str):
    sentence_analyzer = SentimentIntensityAnalyzer()
    temp_sentence = sentence
    sentiment_result = sentence_analyzer.polarity_scores(temp_sentence)
    return (sentiment_result['neg'])

# returns the neutral sentiment intensity for a sentence
def neu_score(sentence: str):
    sentence_analyzer = SentimentIntensityAnalyzer()
    temp_sentence = sentence
    sentiment_result = sentence_analyzer.polarity_scores(temp_sentence)
    return (sentiment_result['neu'])

In [ ]:
# apply pos_score to each tweet
dailyData = wsb.groupby('date').apply(wordsDay)
df =pd.DataFrame({'date':dailyData.index, 'words':dailyData.values})

df['pos_score'] = df['words'].apply(pos_score)
df['neg_score'] = df['words'].apply(neg_score)
df['neu_score'] = df['words'].apply(neu_score)
print(df)


In [ ]:
spy = pd.read_csv('SPY.csv')
spy.rename(columns={'Date':'date'},inplace='True')
spy['date'] = pd.to_datetime(spy['date']).dt.date
data = pd.merge(df,spy, on='date',how='left')
# print(data.columns)
px.scatter(data,y='pos_score',x='Close').show()
# data['Close'].dropna(inplace=True)
data.dropna(subset=['Close'], inplace=True)

# print(data)

Pearson_correllation = pearsonr(data['pos_score'],data['High'])
print(f'Correlation: {Pearson_correllation}')